# Collecting the links

-----------
##### Loading packages

In [2]:
# requires to have selenium and  webdriver_manager installed

# Load packages
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import json
import time
import csv
import pandas as pd
import numpy as np

-----------
##### Navigating to bol.com

In [7]:
# Start Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())

# navigating to bol.com
url = "https://www.bol.com/nl/nl/l/engelse-prentenboek-voor-baby-en-peuterboeken/44325/45783+11209+8292+7419/?rating=all"
driver.get(url)
time.sleep(3)

C:\Users\Admin-HP\AppData\Local\Temp\ipykernel_31684\2154786863.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


-----------
##### Seed (links of the e-books) extraction

In [3]:
base_url = 'https://www.bol.com/nl/nl/l/engelse-prentenboek-voor-baby-en-peuterboeken/44325/45783+11209+8292+7419/?rating=all'
url = base_url
driver.get(url)
time.sleep(1)
soup = BeautifulSoup(driver.page_source)

# extracting link from the first page
def get_url(base_url = 'https://www.bol.com/nl/nl/l/engelse-prentenboek-voor-baby-en-peuterboeken/44325/45783+11209+8292+7419/?rating=all'):
    seed = []
    
    for item in soup.find_all("a",{"data-test":"product-title"}):
        seed.append(item.get('href'))
    return(seed)  
links=get_url()
pages = soup.find_all("a",{"class":"js_pagination_item"})[-1].get_text()

print (str('There are'), int(pages) , str('navigation pages'))
print (str('Every page has'), len(links), str('e-books'))


There are 21 navigation pages
Every page has 22 e-books


In [34]:
# function that iterates through pages 
def get_url(base_url = 'https://www.bol.com/nl/nl/l/engelse-prentenboek-voor-ebook-in-baby-en-peuterboeken/44325/45783+8292+7419/?rating=all',pages =20):
    url = base_url
    counter = 1
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source)
    seed = []
    for page in range(pages):
        for item in soup.find_all("a",{"data-test":"product-title"}):
            seed.append( 'https://www.bol.com' + str(item.get('href')))
        print('printing page '+str(counter))
        try:
            counter= counter + 1
            url= 'https://www.bol.com/nl/nl/l/engelse-prentenboek-voor-ebook-in-baby-en-peuterboeken/44325/45783+8292+7419/?page='+ str(counter)+'&rating=all'
            driver.get(url)
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source)
            # something that block could be added
        except:
            break

    return(seed) 

In [35]:
base_url = 'https://www.bol.com/nl/nl/l/engelse-prentenboek-voor-baby-en-peuterboeken/44325/45783+11209+8292+7419/?rating=all'
url = base_url
counter = 1
driver.get(url)
time.sleep(1)
soup = BeautifulSoup(driver.page_source)

# generating the seed
max_pages = int(soup.find_all("a",{"class":"js_pagination_item"})[-1].get_text())
links=get_url(pages = max_pages)

printing page 1
printing page 2
printing page 3
printing page 4
printing page 5
printing page 6
printing page 7
printing page 8
printing page 9
printing page 10
printing page 11
printing page 12
printing page 13
printing page 14
printing page 15
printing page 16
printing page 17
printing page 18
printing page 19
printing page 20
printing page 21


------------------ 
##### save the seeds in csv

In [45]:
header = ['e-book link']
with open('../data/seed.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for link in links:
        writer.writerows([[link]])
    

# Data collection

------------------ 
##### open the seeds in csv

In [4]:
links =[]
with open('../data/seed.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        links.append(row['e-book link'])


---------------
##### collecting data

this function extract
- title 
- Price (of the paper version)

In [6]:
# this chunk collects title and price of the paper version
links_sample=links[0:7]
data=[]
for link in links_sample:
    url = link
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    dictio= {'title':soup.find('span', {'data-test':"title"}).get_text(),
             'price':soup.find(class_='promo-price').get_text().replace('\n  ','.')[:-1]
            }
    data.append(dictio)
data

[{'title': 'Bing -  Playing Together & Time for Bed (Bing)', 'price': '9.24'},
 {'title': 'Diggersaurs: Mission to Mars', 'price': '6.49'},
 {'title': 'Hey Duggee - Hey Duggee: The Tooth Brushing Badge',
  'price': '4.99'},
 {'title': 'Why I Love My Brother', 'price': '12.20'},
 {'title': 'The Animals of Chinese New Year / 中国农历新年动物生肖 Read-Along',
  'price': '8.99'},
 {'title': "Giraffes Can't Dance", 'price': '2.49'},
 {'title': "Jack's Tractor", 'price': '2.49'}]

In [83]:
# summary regarding the price of the paper version
df = pd.DataFrame.from_dict(data)
clean_df=df.dropna(axis=0)
clean_df = clean_df.astype({'price':'float'})
print('The average is '+str(clean_df['price'].mean()))
print('The minimum is '+str(df['price'].min()))
print('The maximum is '+str(df['price'].max()))

The average is 6.4980000000000055
The minimum is 0.99
The maximum is 9.99


-------------------
The chunk below collects
- title
- price of the e-book
- number of pages (of the paper version)
- other existing versions of the book

In [5]:
links_sample=links[0:7]
data=[]
for link in links_sample:
    url = link
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    try:
        hardcover = soup.find_all("div",{"class":"feature-options feature-options--btns"})
        n_options=len(hardcover[0]('a'))
        #hard=hardcover[0]('a')[1].get('href')

        for option in soup.find_all("div",{"class":"feature-options feature-options--btns"})[0]('a'):
            if 'hard' in str(option):
                hardcover_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(hardcover_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_hard in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_hard.get_text():
                        number_pager_hardcover = float(info_bar_hard.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            else :
                number_pager_hardcover= 'no hardcover'
    except:
        number_pager_hardcover= 'no hardcover'
        n_options = 'no other options'

    dictio= {'title':soup.find('span', {'data-test':"title"}).get_text(),
             'price':float(soup.find(class_='promo-price').get_text().replace('\n  ','.')[:-1]),
             'purchase options': n_options,
             'number of pages': number_pager_hardcover
            }
    data.append(dictio)
data

[{'title': 'Bing -  Playing Together & Time for Bed (Bing)',
  'price': 9.24,
  'purchase options': 'no other options',
  'number of pages': 'no hardcover'},
 {'title': 'Diggersaurs: Mission to Mars',
  'price': 6.49,
  'purchase options': 'no other options',
  'number of pages': 'no hardcover'},
 {'title': 'Hey Duggee - Hey Duggee: The Tooth Brushing Badge',
  'price': 4.99,
  'purchase options': 'no other options',
  'number of pages': 'no hardcover'},
 {'title': 'Why I Love My Brother',
  'price': 12.2,
  'purchase options': 'no other options',
  'number of pages': 'no hardcover'},
 {'title': 'The Animals of Chinese New Year / 中国农历新年动物生肖 Read-Along',
  'price': 8.99,
  'purchase options': 'no other options',
  'number of pages': 'no hardcover'},
 {'title': "Giraffes Can't Dance",
  'price': 14.29,
  'purchase options': 5,
  'number of pages': 'no hardcover'},
 {'title': "Jack's Tractor",
  'price': 2.49,
  'purchase options': 2,
  'number of pages': 'no hardcover'}]

-------------------
The chunk below collects
- title
- price of the e-book
- number of pages (of the hardcover version)
- number of pages (of the paper back version)
- other existing versions of the book

In [8]:
links_sample=links[0:7]
data=[]
for link in links_sample:
    url = link
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    price_ebook = float(soup.find(class_='promo-price').get_text().replace('\n  ','.').replace('-','00')[:-1])
    title = soup.find('span', {'data-test':"title"}).get_text()
    try:
        hardcover = soup.find_all("div",{"class":"feature-options feature-options--btns"})
        n_options=len(hardcover[0]('a'))
        #hard=hardcover[0]('a')[1].get('href')

        for option in soup.find_all("div",{"class":"feature-options feature-options--btns"})[0]('a'):
            if 'ardcover' in str(option):
                hardcover_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(hardcover_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_hard in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_hard.get_text():
                        number_pages_hardcover = int(info_bar_hard.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            elif 'aperback' in str(option):
                paperback_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(paperback_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_paper in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_paper.get_text():
                        number_pages_paperback = int(info_bar_paper.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            else:
                number_pages_paperback = 'no paperback'
                number_pages_hardcover = 'no hardcover'
    except:
        number_pages_hardcover= 'no hardcover'
        number_pages_paperback = 'no paperback'
        n_options = 'no other options'
    

    dictio= {'title': title,
             'price ebook': price_ebook,
             'purchase options': n_options,
             'number of pages (hardcover)': number_pages_hardcover,
             'number of pages (paperback)' : number_pages_paperback
            }
    data.append(dictio)
    print (str(link)+' extracted')
print(len(data),str('books scraped'))

https://www.bol.com/nl/nl/p/bing-playing-together-time-for-bed/9200000043056232/ extracted
https://www.bol.com/nl/nl/p/diggersaurs-mission-to-mars/9200000125004785/ extracted
https://www.bol.com/nl/nl/p/hey-duggee-hey-duggee-the-tooth-brushing-badge/9200000122609976/ extracted
https://www.bol.com/nl/nl/p/why-i-love-my-brother/9200000081819193/ extracted
https://www.bol.com/nl/nl/p/the-animals-of-chinese-new-year-read-along/9200000104504304/ extracted
https://www.bol.com/nl/nl/p/giraffes-can-t-dance/9200000052282205/ extracted
https://www.bol.com/nl/nl/p/jack-s-tractor/9200000061992449/ extracted
7 books scraped


---------
##### most updated scraper
This code extract
- title
- price of the e-book
- number of pages (of the hardcover version)
- number of pages (of the paper back version)
- other existing versions of the book
- ratings
- language of the book

In [21]:
data=[]
for link in links:
    url = link
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    price_ebook = float(soup.find(class_='promo-price').get_text().replace('\n  ','.').replace('-','00')[:-1])
    title = soup.find('span', {'data-test':"title"}).get_text()
    language = soup.find('a', {'data-analytics-tag':"language"}).text.replace('Taal:', '').strip()
    for row in soup.find_all(class_= "specs__row"):
        if 'Leeftijd' in row.get_text(): #Leeftijd means age in dutch
            age = row.find(class_= "specs__value").get_text().replace('\n','').replace('                      ','').strip()
        else:
            0
    try:
        hardcover = soup.find_all("div",{"class":"feature-options feature-options--btns"})
        n_options=len(hardcover[0]('a'))
        #hard=hardcover[0]('a')[1].get('href')

        for option in soup.find_all("div",{"class":"feature-options feature-options--btns"})[0]('a'):
            if 'ardcover' in str(option):
                hardcover_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(hardcover_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_hard in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_hard.get_text():
                        number_pages_hardcover = int(info_bar_hard.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            elif 'aperback' in str(option):
                paperback_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(paperback_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_paper in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_paper.get_text():
                        number_pages_paperback = int(info_bar_paper.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            else:
                number_pages_paperback = 'no paperback'
                number_pages_hardcover = 'no hardcover'
    except:
        number_pages_hardcover= 'no hardcover'
        number_pages_paperback = 'no paperback'
        n_options = 'no other options'
    try:
        reviews = soup.find('div', {'data-test':"rating-suffix"}).get_text()
    except:
        reviews = 'no reviews'
        
        
    

    dictio= {'title': title,
             'price ebook': price_ebook,
             'purchase options': n_options,
             'number of pages (hardcover)': number_pages_hardcover,
             'number of pages (paperback)' : number_pages_paperback,
             'reviews': reviews,
             'language':language,
             'age': age
            }
    data.append(dictio)
    print (str(link)+' extracted')
print(len(data),str('books scraped'))
data

https://www.bol.com/nl/nl/p/bing-playing-together-time-for-bed/9200000043056232/ extracted
https://www.bol.com/nl/nl/p/diggersaurs-mission-to-mars/9200000125004785/ extracted
https://www.bol.com/nl/nl/p/hey-duggee-hey-duggee-the-tooth-brushing-badge/9200000122609976/ extracted
https://www.bol.com/nl/nl/p/why-i-love-my-brother/9200000081819193/ extracted
https://www.bol.com/nl/nl/p/the-animals-of-chinese-new-year-read-along/9200000104504304/ extracted
https://www.bol.com/nl/nl/p/giraffes-can-t-dance/9200000052282205/ extracted
https://www.bol.com/nl/nl/p/jack-s-tractor/9200000061992449/ extracted
https://www.bol.com/nl/nl/p/the-octonauts-and-the-growing-goldfish/9200000025511970/ extracted
https://www.bol.com/nl/nl/p/ten-minutes-to-bed-ten-minutes-to-bed-little-monster/9200000097768460/ extracted
https://www.bol.com/nl/nl/p/baby-loves-science-3-baby-loves-thermodynamics/9200000079763596/ extracted
https://www.bol.com/nl/nl/p/the-fairytale-hairdresser-the-fairytale-hairdresser-and-rapunz

https://www.bol.com/nl/nl/p/topsy-and-tim-have-itchy-heads/9200000035702666/ extracted
https://www.bol.com/nl/nl/p/rain-or-shine/9200000090270549/ extracted
https://www.bol.com/nl/nl/p/what-do-they-do-with-all-that-poo/9200000089410923/ extracted
https://www.bol.com/nl/nl/p/tummy-tom-the-green-collection/9200000028780705/ extracted
https://www.bol.com/nl/nl/p/awesome-engines-big-digger-abc/9200000048011727/ extracted
https://www.bol.com/nl/nl/p/snow-penguin/9200000104593434/ extracted
https://www.bol.com/nl/nl/p/i-can-only-draw-worms/9200000073570126/ extracted
https://www.bol.com/nl/nl/p/first-day-at-bug-school/9200000056087098/ extracted
https://www.bol.com/nl/nl/p/space-tortoise/9200000088336916/ extracted
https://www.bol.com/nl/nl/p/uncle-oscar-s-chairs/9200000092849642/ extracted
https://www.bol.com/nl/nl/p/ten-fat-sausages/9200000082923553/ extracted
https://www.bol.com/nl/nl/p/big-yellow-digger/9200000034182722/ extracted
https://www.bol.com/nl/nl/p/noodle-head/9200000084367876/

[{'title': 'Bing - Playing Together & Time for Bed (Bing)',
  'price ebook': 9.24,
  'purchase options': 'no other options',
  'number of pages (hardcover)': 'no hardcover',
  'number of pages (paperback)': 'no paperback',
  'reviews': 'no reviews',
  'language': 'Engels',
  'age': '0-12 maanden'},
 {'title': 'Diggersaurs: Mission to Mars',
  'price ebook': 6.49,
  'purchase options': 'no other options',
  'number of pages (hardcover)': 'no hardcover',
  'number of pages (paperback)': 'no paperback',
  'reviews': 'no reviews',
  'language': 'Engels',
  'age': '0-12 maanden'},
 {'title': 'Hey Duggee - Hey Duggee: The Tooth Brushing Badge',
  'price ebook': 4.99,
  'purchase options': 'no other options',
  'number of pages (hardcover)': 'no hardcover',
  'number of pages (paperback)': 'no paperback',
  'reviews': 'no reviews',
  'language': 'Engels',
  'age': 'vanaf 1 jaar, vanaf 2 jaar, vanaf 3 jaar'},
 {'title': 'Why I Love My Brother',
  'price ebook': 12.2,
  'purchase options': 'no 

----------
##### saving output in json

In [22]:
with open("../data/dataset", "w") as f:
    json.dump(data, f, indent=4)